In [5]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk, Dataset, DatasetDict
from evaluate import load
import numpy as np
import pandas as pd
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data = pd.read_csv('train.csv')
metric = load('rouge')
model_checkpoints = 'facebook/bart-large-xsum'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [7]:
len(tokenizer.encode(data.abstract.max(), return_tensors='pt')[0])

186

In [8]:
len(tokenizer.encode(data.title.max(), return_tensors='pt')[0])

19

In [9]:
train = Dataset.from_pandas(data[:125000])
val = Dataset.from_pandas(data[125000:])
# test = Dataset.from_pandas(data[130000:])

data = DatasetDict({'train':train, 'validation':val})

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['abstract', 'title'],
        num_rows: 125000
    })
    validation: Dataset({
        features: ['abstract', 'title'],
        num_rows: 10000
    })
})

In [11]:
max_input = 256
max_target = 32

In [12]:
def preprocess_data(data_to_process):
    inputs = [text for text in data_to_process['abstract']]
    model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length', truncation=True)

    with tokenizer.as_target_tokenizer():
        targets = tokenizer(data_to_process['title'], max_length=max_target, padding='max_length', truncation=True)
    
    model_inputs['labels'] = targets['input_ids']
    return model_inputs

In [13]:
tokenized_data = data.map(preprocess_data, batched=True, remove_columns=['abstract', 'title'])

  0%|          | 0/125 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?ba/s]

In [14]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [15]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
batch_size = 12

In [17]:
def compute_rouge(pred):
    predictions, labels = pred
    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
    res = {key: value*100 for key, value in res.items()}

    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)

    return {k: round(v, 4) for k,v in res.items()}

In [18]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    eval_accumulation_steps=1
)

In [19]:
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [20]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 125000
  Num Epochs = 1
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 2
  Total optimization steps = 5208
  Number of trainable parameters = 406290432
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,0.893100,0.967239,47.602100,27.269600,42.364800,42.367300,19.402900


Streaming output truncated to the last 5000 lines.
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,

TrainOutput(global_step=5208, training_loss=0.9763751344929825, metrics={'train_runtime': 15114.4299, 'train_samples_per_second': 8.27, 'train_steps_per_second': 0.345, 'total_flos': 6.771768460679578e+16, 'train_loss': 0.9763751344929825, 'epoch': 1.0})

In [21]:
trainer.save_model("./gt_model")

Saving model checkpoint to ./gt_model
Configuration saved in ./gt_model/config.json
Configuration saved in ./gt_model/generation_config.json
Model weights saved in ./gt_model/pytorch_model.bin
tokenizer config file saved in ./gt_model/tokenizer_config.json
Special tokens file saved in ./gt_model/special_tokens_map.json


In [22]:
test_data = pd.read_csv('test.csv')

In [23]:
texts = test_data.abstract.values

In [25]:
for i in range(len(texts)):
    model_inputs = tokenizer(texts[i], max_length=max_input, padding='max_length', truncation=True)
    pred, _, _ = trainer.predict([model_inputs])
    texts[i] = tokenizer.decode(pred[0], skip_special_tokens=True)

***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



...

***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



***** Running Prediction *****
  Num examples = 1
  Batch size = 12
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "max_length": 62,
  "min_length": 11,
  "no_repeat_ngram_size": 3,
  "num_beams": 6,
  "pad_token_id": 1,
  "transformers_version": "4.26.0"
}



In [26]:
test_data['abstract'] = texts

In [30]:
texts[:5]

array(['attention neural network for sequence transformation',
       'evaluation of the doc2vec approach to document embedding',
       'two-way two-way lstm with conditional random fields',
       'coverage-based attention-based neural machine   translation',
       'unsupervised image-to-image translation by few examples'],
      dtype=object)

In [27]:
test_data

,abstract
0,attention neural network for sequence transfor...
1,evaluation of the doc2vec approach to document...
2,two-way two-way lstm with conditional random f...
3,coverage-based attention-based neural machine ...
4,unsupervised image-to-image translation by few...
...,...
995,subsystem codes derived from mds stabilizer codes
996,dirac-harmonic maps from degenerating spin sur...
997,kloosterman sums twisted by multiplicative cha...
998,long strange segments and ruin probabilities f...


In [28]:
test_data.to_csv('predictions.csv', header=True, index=False)

In [ ]:
path = 'path_to_my_model'
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(path)
tokenizer = transformers.AutoTokenizer.from_pretrained(path)